In [1]:
from fashionData import *
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [2]:
train = FashionData('data/fashion-mnist_train.csv',nrows=1000)

In [3]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x)



In [4]:
network = Net()
print(network)

Net(
  (conv1): Conv2d(1, 10, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(10, 20, kernel_size=(5, 5), stride=(1, 1))
  (conv2_drop): Dropout2d(p=0.5, inplace=False)
  (fc1): Linear(in_features=320, out_features=50, bias=True)
  (fc2): Linear(in_features=50, out_features=10, bias=True)
)


In [5]:
learning_rate = 0.01
optimizer = optim.SGD(network.parameters(), lr=learning_rate)
loss_criterion = nn.CrossEntropyLoss()

In [6]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
n_epochs=10
batch_size = 100


train_loader = torch.utils.data.DataLoader(dataset=train, batch_size=batch_size, shuffle=True)

n_total_steps = len(train_loader)

for epoch in range(n_epochs):
    for i, (labels, images) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        
        # Forward pass and loss calculation
        outputs = network(images)
        loss = loss_criterion(outputs, labels)
        
        # Backward and optimize
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if (i+1) % 100 == 0:
            print (f'Epoch [{epoch+1}/{n_epochs}], Step [{i+1}/{n_total_steps}], Loss: {loss.item():.4f}')

epoch  1  loss =  2.1854541301727295


/var/folders/2y/bnxt2f3n7z5btz_1gx1hdklh0000gn/T/ipykernel_8041/1954227324.py:17: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)


epoch  2  loss =  2.00885272026062
epoch  3  loss =  1.7995036840438843
epoch  4  loss =  1.8642399311065674
epoch  5  loss =  1.5412400960922241
epoch  6  loss =  1.3623939752578735
epoch  7  loss =  1.5196168422698975
epoch  8  loss =  1.2522350549697876
epoch  9  loss =  1.2215213775634766
epoch  10  loss =  1.3229142427444458
